# Dask Delayed

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

In [1]:
import numpy as np
import dask.delayed as delayed
import dask
from time import sleep
import random

## Лабораторная работа 14

1\. Напишите функцию, которая считывает файл формата xml из каталога `reviewers_full` и по данным этого файла формирует список словарей, содержащих следующие ключи: `id`, `username`, `name`, `sex`, `country`, `mail`, `registered`, `birthdate`, `name_prefix`, `country_code`. Часть из этих значений в исходном файле хранится в виде тэгов, часть - в виде атрибутов тэгов. Для конкретного человека какие-то из этих ключей могут отсутствовать. 



In [2]:
from bs4 import BeautifulSoup

In [108]:
import re

In [179]:
def country_code(country):
    pattern = r'"[A-Z]+"'
    if re.findall(pattern, str(country)):
        return re.findall(pattern, str(country))[0][1:-1]

In [120]:
country_code('<country code="DE">Germany</country>')

'DE'

In [213]:
def read_reviewers_full(x):
    with open(
        f'14_delayed_data/reviewers_full/reviewers_full_{x}.xml',
        'r',
        encoding = 'utf-8'    
    ) as fp:
        review = BeautifulSoup(fp)
    
    users = review.find_all('user')
    reviewer = []

    for rev in users:
        
        reviewer.append(
            {
                'id': rev.find_all('id')[0].text if rev.find_all('id') else None,
                'username': rev.find_all('username')[0].text if rev.find_all('username') else None,
                'name': rev.find_all('name')[0].text if rev.find_all('name') else None,
                'sex': rev.find_all('sex')[0].text if rev.find_all('sex') else None,
                'country': rev.find_all('country')[0].text if rev.find_all('country') else None,
                'mail': rev.find_all('mail')[0].text if rev.find_all('mail') else None,
                'registered': rev.find_all('registered')[0].text if rev.find_all('registered') else None,
                'birthdate': rev.find_all('birthdate')[0].text if rev.find_all('birthdate') else None,
                'name_prefix': rev.attrs['prefix'] if 'prefix' in rev else None,
                'country_code': country_code(rev.find_all('country')[0] if rev.find_all('country') else None)
            }
        )
    
    return reviewer

In [214]:
read_reviewers_full(0)

[{'id': '88005',
  'username': 'jacqueline00',
  'name': 'Michele Lewis',
  'sex': None,
  'country': None,
  'mail': 'morenocharlotte@yahoo.com',
  'registered': None,
  'birthdate': None,
  'name_prefix': None,
  'country_code': None},
 {'id': '68591',
  'username': 'daniellegomez',
  'name': None,
  'sex': 'F',
  'country': 'Germany',
  'mail': None,
  'registered': None,
  'birthdate': '2005-03-06',
  'name_prefix': None,
  'country_code': 'DE'},
 {'id': '81003',
  'username': 'alucero',
  'name': 'Tammy Patton',
  'sex': None,
  'country': None,
  'mail': 'larsenrobert@gmail.com',
  'registered': None,
  'birthdate': None,
  'name_prefix': None,
  'country_code': None},
 {'id': '61509',
  'username': 'mcleanjacqueline',
  'name': 'Jeremy Schmidt',
  'sex': 'M',
  'country': None,
  'mail': None,
  'registered': '2017-09-21',
  'birthdate': None,
  'name_prefix': None,
  'country_code': None},
 {'id': '74667',
  'username': 'wesley96',
  'name': None,
  'sex': None,
  'country': No

2\. Измерьте время выполнения функции из задания 1 на всех файлах из каталога `reviewers_full`. Ускорьте время выполнения, используя `dask.delayed`.

In [210]:
%%time
for i in range(20):
    read_reviewers_full(i)

CPU times: user 8min 16s, sys: 4min 27s, total: 12min 44s
Wall time: 23min 47s


In [211]:
dask_read_reviewers_full = dask.delayed(read_reviewers_full)

In [212]:
%%time
r = [dask_read_reviewers_full(i) for i in range(20)]
r = dask.compute(r, scheduler = 'multiprocessing')

/Users/nastya/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/Users/nastya/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` i

CPU times: user 1.35 s, sys: 1.13 s, total: 2.48 s
Wall time: 3min 3s


3\. Задекорируйте функцию из задания 1 при помощи `dask.delayed` и создайте список `reviewers`, состоящий из 5 объектов `delayed` (по одному объекту на файл). Из списка объектов `delayed`, создайте `dask.bag` при помощи метода `db.from_delayed`. Добавьте ключ `birth_year`, в котором хранится год рождения человека. Оставьте в выборке только тех людей, которые __наверняка__ моложе 1980 года. Преобразуйте поле `id` к целому типу.

In [215]:
@dask.delayed
def read_reviewers_full(x):
    with open(
        f'14_delayed_data/reviewers_full/reviewers_full_{x}.xml',
        'r',
        encoding = 'utf-8'    
    ) as fp:
        review = BeautifulSoup(fp)
    
    users = review.find_all('user')
    reviewer = []

    for rev in users:
        
        reviewer.append(
            {
                'id': rev.find_all('id')[0].text if rev.find_all('id') else None,
                'username': rev.find_all('username')[0].text if rev.find_all('username') else None,
                'name': rev.find_all('name')[0].text if rev.find_all('name') else None,
                'sex': rev.find_all('sex')[0].text if rev.find_all('sex') else None,
                'country': rev.find_all('country')[0].text if rev.find_all('country') else None,
                'mail': rev.find_all('mail')[0].text if rev.find_all('mail') else None,
                'registered': rev.find_all('registered')[0].text if rev.find_all('registered') else None,
                'birthdate': rev.find_all('birthdate')[0].text if rev.find_all('birthdate') else None,
                'name_prefix': rev.attrs['prefix'] if 'prefix' in rev else None,
                'country_code': country_code(rev.find_all('country')[0] if rev.find_all('country') else None)
            }
        )
    
    return reviewer

In [216]:
%%time
reviewers = [read_reviewers_full(i) for i in range(20)]

CPU times: user 4.19 ms, sys: 7.91 ms, total: 12.1 ms
Wall time: 13.9 ms


In [217]:
import dask.bag as db

In [218]:
bag_from_reviewers = db.from_delayed(reviewers)

In [219]:
bag_from_reviewers.take(2)

/Users/nastya/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


({'id': '88005',
  'username': 'jacqueline00',
  'name': 'Michele Lewis',
  'sex': None,
  'country': None,
  'mail': 'morenocharlotte@yahoo.com',
  'registered': None,
  'birthdate': None,
  'name_prefix': None,
  'country_code': None},
 {'id': '68591',
  'username': 'daniellegomez',
  'name': None,
  'sex': 'F',
  'country': 'Germany',
  'mail': None,
  'registered': None,
  'birthdate': '2005-03-06',
  'name_prefix': None,
  'country_code': 'DE'})

In [220]:
from datetime import datetime

In [274]:
def year(review):
    review['id'] = (int(review['id']) if review['id'] else None)
    try:
        review['birth_year'] = datetime.strptime(review['birthdate'], '%Y-%m-%d').year
    except:
        review['birth_year'] = -1
    
    return review

In [275]:
bag_from_reviewers.map(year).take(2)

/Users/nastya/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


({'id': 88005,
  'username': 'jacqueline00',
  'name': 'Michele Lewis',
  'sex': None,
  'country': None,
  'mail': 'morenocharlotte@yahoo.com',
  'registered': None,
  'birthdate': None,
  'name_prefix': None,
  'country_code': None,
  'birth_year': -1},
 {'id': 68591,
  'username': 'daniellegomez',
  'name': None,
  'sex': 'F',
  'country': 'Germany',
  'mail': None,
  'registered': None,
  'birthdate': '2005-03-06',
  'name_prefix': None,
  'country_code': 'DE',
  'birth_year': 2005})

In [276]:
bag_from_reviewers_years = bag_from_reviewers.map(year)

In [277]:
bag_from_reviewers_years = bag_from_reviewers_years.filter(
    lambda p: p['birth_year'] > 1980
)

In [278]:
bag_from_reviewers_years.take(2)

/Users/nastya/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


({'id': 68591,
  'username': 'daniellegomez',
  'name': None,
  'sex': 'F',
  'country': 'Germany',
  'mail': None,
  'registered': None,
  'birthdate': '2005-03-06',
  'name_prefix': None,
  'country_code': 'DE',
  'birth_year': 2005},
 {'id': 43614,
  'username': 'jeffreynelson',
  'name': None,
  'sex': 'F',
  'country': 'Aruba',
  'mail': 'laura77@hotmail.com',
  'registered': '2019-11-19',
  'birthdate': '1994-10-10',
  'name_prefix': None,
  'country_code': 'AW',
  'birth_year': 1994})